## 1.2 LCEL(LangChain Expression Language)

<div style="text-align: right"> Initial issue : 2025.04.26 </div>
<div style="text-align: right"> last update : 2025.04.26 </div>

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합가능

```
chain = prompt | model | output_parser
```

- `|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달.

- 이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됨. 
- 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있음.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from utils import langsmith

langsmith(project_name="ch01")

LangSmith 추적을 시작합니다.
[프로젝트명]
ch01


In [3]:
from utils import stream_response

### 프롬프트 템플릿

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [4]:
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성

In [5]:
template = "{country}의 수도는 무엇인가요?"
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 무엇인가요?')

In [6]:
prompt = prompt_template.format(country="한국")
prompt

'한국의 수도는 무엇인가요?'

In [7]:
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 무엇인가요?'

In [8]:
prompt_template.invoke(input={"country": "한국"})

StringPromptValue(text='한국의 수도는 무엇인가요?')

### chain 생성

In [9]:
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model

In [10]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x165944450>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x166025d90>, root_client=<openai.OpenAI object at 0x164979bd0>, root_async_client=<openai.AsyncOpenAI object at 0x166020590>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

invoke 호출
- python dictionary 형태로 값을 전달

In [11]:
input = {"topic": "인공지능 모델의 학습 원리"}

In [12]:
chain.invoke(input=input)

AIMessage(content='인공지능 모델의 학습 원리를 쉽게 설명하자면, 사람의 학습 과정과 비슷하다고 할 수 있습니다. 다음과 같은 단계로 설명할 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다. 예를 들어, 고양이와 개를 구분하는 모델을 만들고 싶다면, 고양이와 개의 사진이 포함된 데이터셋이 필요합니다.\n\n2. **훈련**: 모델은 수집한 데이터를 바탕으로 학습을 시작합니다. 이 과정에서 모델은 입력 데이터(예: 고양이 사진)를 받고, 그에 대한 정답(예: "고양이" 또는 "개")을 비교합니다. 모델은 처음에는 잘못된 예측을 할 수 있지만, 점차적으로 정답에 가까워지도록 조정됩니다.\n\n3. **오류 수정**: 모델이 잘못된 예측을 했을 때, 그 오류를 분석하고 수정하는 과정을 거칩니다. 이를 위해 \'손실 함수\'라는 수학적 방법을 사용하여 예측과 실제 정답 간의 차이를 계산합니다. 이 차이를 줄이기 위해 모델의 내부 파라미터(가중치)를 조정합니다.\n\n4. **반복 학습**: 이 과정은 여러 번 반복됩니다. 모델은 많은 데이터를 통해 점점 더 정확한 예측을 할 수 있도록 학습합니다. 이 과정을 \'에포크\'라고 하며, 여러 번 반복할수록 모델의 성능이 향상됩니다.\n\n5. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(훈련에 사용되지 않은 데이터)를 사용합니다. 이를 통해 모델이 실제 상황에서도 잘 작동하는지 확인합니다.\n\n6. **배포**: 모델이 충분히 학습하고 검증되면, 실제 애플리케이션에 배포되어 사용됩니다. 예를 들어, 사진을 올리면 그 사진이 고양이인지 개인지 자동으로 판단해주는 앱이 될 수 있습니다.\n\n이렇게 인공지능 모델은 데이터를 통해 학습하고, 반복적인 과정을 통해 점점 더 똑똑해지는 것입니다.', additional_kwargs={'refusal': None}

stream 출력

In [13]:
answer = chain.stream(input)
stream_response(answer)

인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 이해할 수 있습니다.

1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.

2. **데이터 전처리**: 수집한 데이터는 모델이 이해할 수 있는 형태로 가공해야 합니다. 예를 들어, 이미지의 크기를 조정하거나, 텍스트를 숫자로 변환하는 과정이 필요합니다.

3. **모델 선택**: 학습할 모델을 선택합니다. 예를 들어, 이미지 인식에는 CNN(합성곱 신경망)을, 자연어 처리에는 RNN(순환 신경망)이나 Transformer 모델을 사용할 수 있습니다.

4. **학습**: 모델은 주어진 데이터를 바탕으로 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 정답(라벨)을 비교하여 오차를 계산하고, 이 오차를 줄이기 위해 가중치를 조정합니다. 이 과정을 반복하면서 모델은 점점 더 정확하게 예측할 수 있게 됩니다.

5. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(검증 데이터)를 사용합니다. 이 데이터는 모델이 학습할 때 사용하지 않은 데이터로, 모델의 일반화 능력을 확인하는 데 중요합니다.

6. **조정 및 개선**: 검증 결과에 따라 모델의 구조나 하이퍼파라미터(학습률, 배치 크기 등)를 조정하여 성능을 개선할 수 있습니다.

7. **배포**: 최종적으로 학습된 모델은 실제 환경에서 사용될 수 있도록 배포됩니다. 이때, 사용자는 모델이 제공하는 예측이나 결과를 활용하게 됩니다.

이러한 과정을 통해 인공지능 모델은 데이터를 기반으로 학습하고, 새로운 상황에서도 유용한 예측을 할 수 있게 됩니다.

출력 파서(OutputParser)

In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [15]:
chain = prompt | model | output_parser

In [16]:
chain.invoke({"topic": "인공지능 모델의 학습 원리"})

"인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 이해할 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 예를 들어, 고양이와 개를 구분하는 모델을 만들고 싶다면, 고양이와 개의 사진을 많이 모아야 합니다.\n\n2. **데이터 전처리**: 수집한 데이터는 모델이 이해할 수 있는 형태로 가공해야 합니다. 예를 들어, 이미지의 크기를 조정하거나, 색상을 표준화하는 등의 작업을 합니다.\n\n3. **모델 선택**: 학습할 모델의 종류를 선택합니다. 예를 들어, 신경망, 결정 트리, 서포트 벡터 머신 등 다양한 알고리즘 중에서 선택할 수 있습니다.\n\n4. **학습**: 모델에 데이터를 입력하고, 정답(레이블)과 비교하여 오차를 계산합니다. 이 오차를 줄이기 위해 모델의 내부 파라미터를 조정하는 과정을 반복합니다. 이 과정을 '훈련'이라고 합니다.\n\n5. **검증**: 학습이 끝난 후, 모델이 얼마나 잘 학습했는지 확인하기 위해 새로운 데이터(검증 데이터)를 사용하여 성능을 평가합니다.\n\n6. **예측**: 모델이 잘 학습되었다면, 이제 새로운 데이터에 대해 예측을 할 수 있습니다. 예를 들어, 새로운 사진이 고양이인지 개인지 판단할 수 있습니다.\n\n7. **반복 개선**: 모델의 성능이 만족스럽지 않다면, 데이터나 모델 구조를 변경하거나, 하이퍼파라미터를 조정하여 다시 학습하는 과정을 반복합니다.\n\n이러한 과정을 통해 인공지능 모델은 주어진 데이터를 기반으로 패턴을 학습하고, 새로운 데이터에 대해 예측할 수 있는 능력을 갖추게 됩니다."

In [17]:
answer = chain.stream(input)
stream_response(answer)

인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.

1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다. 예를 들어, 고양이와 개를 구분하는 모델을 만들고 싶다면, 고양이와 개의 사진이 포함된 데이터셋이 필요합니다.

2. **데이터 전처리**: 수집한 데이터는 종종 정리하고 변환해야 합니다. 예를 들어, 이미지의 크기를 맞추거나, 텍스트를 숫자로 변환하는 등의 작업이 필요합니다. 이 과정을 통해 모델이 데이터를 더 잘 이해할 수 있도록 돕습니다.

3. **모델 선택**: 인공지능에는 다양한 종류의 모델이 있습니다. 예를 들어, 신경망, 결정 트리, 서포트 벡터 머신 등이 있습니다. 문제의 특성에 맞는 모델을 선택합니다.

4. **학습**: 모델은 데이터를 통해 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터(예: 고양이 사진)와 정답(예: '고양이')을 비교하여, 얼마나 잘 맞추는지를 평가합니다. 이 평가 결과를 바탕으로 모델의 내부 파라미터를 조정하여 점점 더 정확하게 예측할 수 있도록 합니다.

5. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(훈련에 사용되지 않은 데이터)를 사용합니다. 이를 통해 모델이 실제 상황에서도 잘 작동하는지를 확인합니다.

6. **배포 및 사용**: 모델이 충분히 잘 학습되었다면, 실제 애플리케이션에 배포하여 사용합니다. 예를 들어, 고양이와 개를 구분하는 앱에 모델을 적용할 수 있습니다.

7. **모델 개선**: 시간이 지나면서 모델의 성능을 지속적으로 모니터링하고, 필요에 따라 추가 데이터를 학습시키거나 모델을 업데이트하여 성능을 개선할 수 있습니다.

이러한 과정을 통해 인공지능 모델은 데이터를 기반으로 학습하고, 새로운 상황에서도 유용한 예측을 할 수 있게 됩니다.

### template 사용하기

In [18]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [19]:
chain = prompt | model | output_parser

In [20]:
print(chain.invoke({"question": "저는 식당에 가서 음식을 준비하고 싶어요."}))

- 영어 회화:  
**You:** Hi there! I would like to place an order for takeout, please.  
**Waiter:** Sure! What would you like to order?  
**You:** I’ll have the grilled chicken salad and a side of garlic bread.  
**Waiter:** Great choice! Would you like any drinks with that?  
**You:** Yes, I’d like a bottle of water, please.  
**Waiter:** Perfect! Your order will be ready shortly.  

- 한글 해석:  
**당신:** 안녕하세요! 포장 주문을 하고 싶습니다.  
**웨이터:** 알겠습니다! 무엇을 주문하시겠습니까?  
**당신:** 구운 치킨 샐러드와 마늘빵 하나 주세요.  
**웨이터:** 좋은 선택이십니다! 음료수는 필요하신가요?  
**당신:** 네, 생수 한 병 주세요.  
**웨이터:** 완벽합니다! 주문은 곧 준비될 것입니다.


In [21]:
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
stream_response(answer)

- 영어 회화:  
A: Hello! Welcome to [Restaurant Name]. How many people are in your party?  
B: Hi! It’s just me. Can I please have a table for one?  
A: Of course! Right this way. Here’s your menu.  
B: Thank you! I’ll start with the soup of the day, and for my main course, I’d like the grilled chicken salad.  
A: Great choice! Would you like anything to drink?  
B: Yes, I’ll have a glass of water, please.  
A: Sure! I’ll get that for you right away.  

- 한글 해석:  
A: 안녕하세요! [식당 이름]에 오신 것을 환영합니다. 몇 분이 함께 하시나요?  
B: 안녕하세요! 저 혼자입니다. 1인용 테이블 하나 부탁드립니다.  
A: 알겠습니다! 이쪽으로 오세요. 여기 메뉴입니다.  
B: 감사합니다! 먼저 오늘의 수프를 주문할게요. 그리고 메인 코스로 그릴드 치킨 샐러드를 주문하고 싶어요.  
A: 좋은 선택이에요! 음료는 무엇을 드릴까요?  
B: 네, 물 한 잔 부탁드립니다.  
A: 알겠습니다! 즉시 가져다 드리겠습니다.